# Modelo Preditivo de Safras Agricolas

## Demonstracao do Projeto 3 - Portfolio Data Science

Este notebook demonstra o pipeline de Machine Learning para previsao de rendimento agricola:

- **Analise Exploratoria**: Visualizacao dos dados de safras
- **Feature Engineering**: Criacao de variaveis preditivas
- **Treinamento**: Comparacao de modelos de ML
- **Avaliacao**: Metricas e interpretacao

### Variavel Alvo
**Rendimento (kg/ha)**: Producao por hectare plantado

---

In [ ]:
# Imports
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

CORES = {
    'primaria': '#2E7D32',
    'secundaria': '#FFA000',
    'destaque': '#1976D2',
    'erro': '#D32F2F'
}

print('Bibliotecas carregadas!')

## 1. Gerando Dados de Safras

Dados baseados em padroes da PAM/IBGE.

In [ ]:
# Configuracoes
anos = list(range(2010, 2024))
estados = ['MT', 'PR', 'RS', 'GO', 'MS', 'SP', 'MG', 'BA', 'SC', 'PI']
culturas = ['Soja', 'Milho', 'Arroz', 'Feijao', 'Cana-de-acucar']

# Rendimentos medios por cultura (kg/ha) - baseados em dados reais
rendimento_base = {
    'Soja': 3200,
    'Milho': 5500,
    'Arroz': 6000,
    'Feijao': 1200,
    'Cana-de-acucar': 75000
}

# Gerar dados
records = []
for ano in anos:
    for estado in estados:
        for cultura in culturas:
            # Rendimento com tendencia de crescimento e variacao
            base = rendimento_base[cultura]
            tendencia = 1 + 0.02 * (ano - 2010)  # 2% ao ano
            variacao_estado = np.random.uniform(0.8, 1.2)
            variacao_ano = np.random.uniform(0.9, 1.1)
            
            rendimento = base * tendencia * variacao_estado * variacao_ano
            
            # Area plantada (hectares)
            area_base = np.random.uniform(50000, 500000)
            area = area_base * variacao_estado
            
            records.append({
                'ano': ano,
                'estado': estado,
                'cultura': cultura,
                'area_plantada_ha': round(area, 0),
                'rendimento_kg_ha': round(rendimento, 2)
            })

df = pd.DataFrame(records)
df['producao_ton'] = (df['area_plantada_ha'] * df['rendimento_kg_ha']) / 1000

print(f'Dataset gerado: {len(df)} registros')
print(f'Periodo: {df["ano"].min()} - {df["ano"].max()}')
print(f'Culturas: {df["cultura"].nunique()}')
print(f'Estados: {df["estado"].nunique()}')
df.head()

## 2. Analise Exploratoria

In [ ]:
# Estatisticas por cultura
stats = df.groupby('cultura').agg({
    'rendimento_kg_ha': ['mean', 'std', 'min', 'max'],
    'area_plantada_ha': 'sum',
    'producao_ton': 'sum'
}).round(2)

stats.columns = ['Rend. Medio', 'Rend. Std', 'Rend. Min', 'Rend. Max', 'Area Total', 'Producao Total']
stats

In [ ]:
# Evolucao do rendimento por cultura
df_evolucao = df.groupby(['ano', 'cultura'])['rendimento_kg_ha'].mean().reset_index()

fig = px.line(
    df_evolucao,
    x='ano',
    y='rendimento_kg_ha',
    color='cultura',
    markers=True
)

fig.update_layout(
    title={'text': 'Evolucao do Rendimento por Cultura', 'x': 0.5, 'font': {'size': 18}},
    xaxis_title='Ano',
    yaxis_title='Rendimento (kg/ha)',
    legend_title='Cultura',
    height=450
)

fig.show()

In [ ]:
# Box plot por estado (Soja)
df_soja = df[df['cultura'] == 'Soja']

fig = px.box(
    df_soja,
    x='estado',
    y='rendimento_kg_ha',
    color='estado',
    color_discrete_sequence=px.colors.qualitative.Set2
)

fig.update_layout(
    title={'text': 'Distribuicao do Rendimento de Soja por Estado', 'x': 0.5, 'font': {'size': 18}},
    xaxis_title='Estado',
    yaxis_title='Rendimento (kg/ha)',
    showlegend=False,
    height=400
)

fig.show()

## 3. Feature Engineering

Criando variaveis preditivas sem data leakage.

In [ ]:
# Preparar features
def create_features(df):
    """Cria features para o modelo sem data leakage."""
    df = df.copy()
    
    # Ordenar por cultura, estado e ano
    df = df.sort_values(['cultura', 'estado', 'ano']).reset_index(drop=True)
    
    # Lag features (rendimento dos anos anteriores)
    for lag in [1, 2, 3]:
        df[f'rendimento_lag{lag}'] = df.groupby(['cultura', 'estado'])['rendimento_kg_ha'].shift(lag)
    
    # Media movel dos ultimos 3 anos (excluindo atual)
    df['rendimento_ma3'] = df.groupby(['cultura', 'estado'])['rendimento_kg_ha'].transform(
        lambda x: x.shift(1).rolling(3, min_periods=1).mean()
    )
    
    # Taxa de crescimento baseada em lags
    df['growth_rate'] = (df['rendimento_lag1'] - df['rendimento_lag2']) / df['rendimento_lag2']
    
    # Interacao area x rendimento historico
    df['area_x_rend_hist'] = df['area_plantada_ha'] * df['rendimento_lag1']
    
    # Remover linhas com NaN (primeiros anos sem lags)
    df = df.dropna()
    
    return df

df_features = create_features(df)
print(f'Registros apos feature engineering: {len(df_features)}')
print(f'\nFeatures criadas:')
print('  - rendimento_lag1/2/3: Rendimento dos anos anteriores')
print('  - rendimento_ma3: Media movel 3 anos')
print('  - growth_rate: Taxa de crescimento')
print('  - area_x_rend_hist: Interacao area x historico')

In [ ]:
# Matriz de correlacao
features_numericas = ['area_plantada_ha', 'rendimento_lag1', 'rendimento_lag2', 
                       'rendimento_lag3', 'rendimento_ma3', 'rendimento_kg_ha']

corr = df_features[features_numericas].corr()

fig = px.imshow(
    corr,
    text_auto='.2f',
    color_continuous_scale='RdBu_r',
    zmin=-1, zmax=1
)

fig.update_layout(
    title={'text': 'Matriz de Correlacao das Features', 'x': 0.5, 'font': {'size': 18}},
    height=450
)

fig.show()

## 4. Treinamento de Modelos

In [ ]:
# Preparar dados para treino
feature_cols = ['area_plantada_ha', 'rendimento_lag1', 'rendimento_lag2', 
                'rendimento_lag3', 'rendimento_ma3', 'growth_rate']

# Remover infinitos e NaN
df_ml = df_features.replace([np.inf, -np.inf], np.nan).dropna(subset=feature_cols + ['rendimento_kg_ha'])

X = df_ml[feature_cols]
y = df_ml['rendimento_kg_ha']

# Split temporal (treino: ate 2020, teste: 2021+)
train_mask = df_ml['ano'] <= 2020
X_train, X_test = X[train_mask], X[~train_mask]
y_train, y_test = y[train_mask], y[~train_mask]

print(f'Treino: {len(X_train)} registros (ate 2020)')
print(f'Teste: {len(X_test)} registros (2021+)')

In [ ]:
# Treinar modelos
modelos = {
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(alpha=1.0),
    'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42)
}

resultados = []

for nome, modelo in modelos.items():
    # Treinar
    modelo.fit(X_train, y_train)
    
    # Prever
    y_pred = modelo.predict(X_test)
    
    # Calcular metricas
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    
    resultados.append({
        'Modelo': nome,
        'RMSE': round(rmse, 2),
        'MAE': round(mae, 2),
        'R2': round(r2, 4),
        'MAPE': round(mape, 2)
    })
    
    print(f'{nome}: RMSE={rmse:.2f}, R2={r2:.4f}')

df_resultados = pd.DataFrame(resultados).sort_values('R2', ascending=False)
df_resultados

## 5. Visualizacao dos Resultados

In [ ]:
# Comparacao de modelos
fig = make_subplots(rows=1, cols=2, subplot_titles=('R2 (Coef. Determinacao)', 'RMSE (Erro)'))

# R2
fig.add_trace(
    go.Bar(
        x=df_resultados['Modelo'],
        y=df_resultados['R2'],
        marker_color=CORES['primaria'],
        text=df_resultados['R2'].apply(lambda x: f'{x:.3f}'),
        textposition='outside',
        name='R2'
    ),
    row=1, col=1
)

# RMSE
fig.add_trace(
    go.Bar(
        x=df_resultados['Modelo'],
        y=df_resultados['RMSE'],
        marker_color=CORES['erro'],
        text=df_resultados['RMSE'].apply(lambda x: f'{x:.0f}'),
        textposition='outside',
        name='RMSE'
    ),
    row=1, col=2
)

fig.update_layout(
    title={'text': 'Comparacao de Modelos', 'x': 0.5, 'font': {'size': 18}},
    height=400,
    showlegend=False
)

fig.update_xaxes(tickangle=-45)
fig.show()

In [ ]:
# Predicao vs Real (melhor modelo)
melhor_modelo = modelos['Gradient Boosting']
y_pred_best = melhor_modelo.predict(X_test)

fig = px.scatter(
    x=y_test,
    y=y_pred_best,
    labels={'x': 'Valor Real (kg/ha)', 'y': 'Valor Predito (kg/ha)'},
    opacity=0.6
)

# Linha de referencia
fig.add_trace(go.Scatter(
    x=[y_test.min(), y_test.max()],
    y=[y_test.min(), y_test.max()],
    mode='lines',
    line=dict(color='red', dash='dash'),
    name='Perfeito'
))

fig.update_layout(
    title={'text': 'Gradient Boosting: Predicao vs Valor Real', 'x': 0.5, 'font': {'size': 18}},
    height=450
)

fig.show()

In [ ]:
# Feature Importance (Gradient Boosting)
importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': melhor_modelo.feature_importances_
}).sort_values('importance', ascending=True)

fig = px.bar(
    importance,
    x='importance',
    y='feature',
    orientation='h',
    color='importance',
    color_continuous_scale='Greens'
)

fig.update_layout(
    title={'text': 'Importancia das Features (Gradient Boosting)', 'x': 0.5, 'font': {'size': 18}},
    xaxis_title='Importancia',
    yaxis_title='',
    height=350,
    coloraxis_showscale=False
)

fig.show()

## 6. Conclusoes

In [ ]:
# Melhor resultado
melhor = df_resultados.iloc[0]

print('=' * 60)
print('RESUMO DO MODELO PREDITIVO DE SAFRAS')
print('=' * 60)

print(f'''
DADOS
   Registros: {len(df):,}
   Culturas: {df['cultura'].nunique()}
   Estados: {df['estado'].nunique()}
   Periodo: {df['ano'].min()}-{df['ano'].max()}

MELHOR MODELO: {melhor['Modelo']}
   R2: {melhor['R2']:.4f} ({melhor['R2']*100:.1f}% da variancia explicada)
   RMSE: {melhor['RMSE']:.2f} kg/ha
   MAE: {melhor['MAE']:.2f} kg/ha
   MAPE: {melhor['MAPE']:.2f}%

FEATURES MAIS IMPORTANTES
   1. rendimento_lag1 (rendimento do ano anterior)
   2. rendimento_ma3 (media movel 3 anos)
   3. rendimento_lag2 (rendimento 2 anos atras)

VALIDACAO
   Metodo: Split temporal (treino ate 2020, teste 2021+)
   Sem data leakage confirmado
''')
print('=' * 60)

---

## Proximos Passos

**Para executar o pipeline completo:**

```bash
# Pipeline completo
python main.py

# Apenas treinamento
python main.py --train

# Fazer predicoes
python main.py --predict --model-path data/models/model.pkl
```

---

*Desenvolvido como parte do Portfolio de Data Science*